# Implementing custom model with dataset and train it for sentiment analysis

In [11]:
!pip install transformers datasets torch
!pip install --upgrade torch torchvision torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.6 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 8.9.2.26
    Uninstalling nvidia-cudnn-cu12-8.9.2.26:
      Successfully uninstalled nvidia-cudnn-cu12-8.9.2.26
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled torch-2.3.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.

In [12]:
#load and prepare the data

from datasets import load_dataset

# Load the IMDb dataset
dataset = load_dataset("imdb")
print(dataset.__class__)#dictonary
print(dataset.column_names)

# Split the dataset into training and test sets
train_dataset = dataset['train']
test_dataset = dataset['test']

print(train_dataset)


<class 'datasets.dataset_dict.DatasetDict'>
{'train': ['text', 'label'], 'test': ['text', 'label'], 'unsupervised': ['text', 'label']}
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [13]:
#preprocessing the data

from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


RuntimeError: THPDtypeType.tp_dict == nullptr INTERNAL ASSERT FAILED at "../torch/csrc/Dtype.cpp":176, please report a bug to PyTorch. 

In [ ]:
# Prepare the Data for Training

from torch.utils.data import DataLoader

# Convert datasets to PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)


In [ ]:
# create the model

from transformers import BertForSequenceClassification

# Load the BERT model with a classification head
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#  Train the Model

from transformers import AdamW
from torch.optim.lr_scheduler import StepLR
import torch

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=0.01, no_deprecation_warning=True)

# Set up the learning rate scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

# Set up the device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
model.train()
for epoch in range(3):  # Train for 3 epochs
    for batch in train_dataloader:
        # Move data to the device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Separate labels from input data
        labels = batch.pop('label')

        # Forward pass
        outputs = model(**batch, labels=labels) # Pass labels to the model separately
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Update learning rate
    scheduler.step()

    print(f"Epoch {epoch + 1} completed. Loss: {loss.item()}")

KeyboardInterrupt: 

In [ ]:
# Evaluate the Model

from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        logits = outputs.logits
        predictions.append(torch.argmax(logits, dim=-1).cpu().numpy())
        true_labels.append(batch["labels"].cpu().numpy())

# Flatten lists and calculate accuracy
predictions = [item for sublist in predictions for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")



NameError: name 'model' is not defined